<a href="https://colab.research.google.com/github/ayushpatra11/RLU_UoM/blob/main/COM64102_Assignment_a85716ap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment Brief: Probabilistic Programming and Active Learning

## Deadline: May 7, 2025, 17:00 GMT

## Number of marks available: 50

This coursework is made of two parts. In the first part, you will use probabilistic programming to perform Bayesian inference in different models. In the second part, you will use uncertainty to guide datapoint selection to create a dataset to train a machine learning model.

### Please READ the whole assignment first, before starting to work on it.

### How and what to submit

A. A **Jupyter Notebook** with the code in all the cells executed and outputs displayed.

B. Name your Notebook **COM64102_Assignment_XXXXXX.ipynb** where XXXXXX is your username such as such as abc18de. Example: `COM64102_Assignment_abc18de.ipynb`

C. Upload the Jupyter Notebook in B to Blackboard under the submission area before the deadline.

D. **NO DATA UPLOAD**: Please do not upload the data files used in this Notebook. We have a copy already.


### Assessment Criteria

* Being able to compute the posterior distribution in Bayesian logistic regression using a probabilistic program.

* Being able to compute several acquisition functions useful for active learning.

* Being able to use active learning to speed-up the training process of a digit classifier.


### Code quality and use of Python libraries
When writing your code, you will find out that there are operations that are repeated at least twice. If your code is unreadable, we may not award marks for that section. Make sure to check the following:

* Did you include Python functions to solve the question and avoid repeating code?
* Did you comment your code to make it readable to others?

### Late submissions

We follow Department's guidelines about late submissions, i.e., a deduction of 10% of the mark each 24 hours the work is late after the deadline. NO late submission will be marked one week after the deadline.

### Use of unfair means

**Any form of unfair means is treated as a serious academic offence and action may be taken under the Discipline Regulations.** Please carefully read [what constitutes Unfair Means](https://documents.manchester.ac.uk/display.aspx?DocID=2870) if not sure. If you still have questions, please ask your Personal tutor or the Lecturers.

# 1. Probabilistic Programming with PyMC

[PyMC](https://www.pymc.io/) is a powerful Python library for **Bayesian statistical modeling** and **probabilistic machine learning**. It provides an intuitive and flexible way to define complex models using a **probabilistic programming framework**.

PyMC makes it easy to express statistical models in terms of probability distributions, and it automates the process of computing posterior distributions using state-of-the-art Markov Chain Monte Carlo (MCMC) sampling methods like:
- [**No-U-Turn Sampler (NUTS)**](https://jmlr.org/papers/volume15/hoffman14a/hoffman14a.pdf), an efficient Hamiltonian Monte Carlo method.
- **Metropolis-Hastings** ‚ a traditional and widely-used MCMC approach.

Bayesian inference involves calculating the posterior distribution:
$$
P(\theta | X) \propto P(X | \theta)P(\theta)
$$
where:
- $P(\theta)$‚ a prior distribution (our prior belief about the parameters).
- $P(X | \theta)$‚ a likelihood of the data given the parameters.
- $P(\theta | X)$, a posterior distribution (what we want to estimate).

Computing this posterior distribution analytically can be difficult or even impossible for complex models. PyMC solves this problem by using efficient numerical sampling techniques, allowing you to approximate the posterior distribution and extract meaningful insights.

## How Does PyMC Work?
1. **Model Definition**: Define the model using PyMC probabilistic building blocks (e.g., `pm.Normal`, `pm.Bernoulli`).
2. **Sampling**: Use `pm.sample()` to draw samples from the posterior distribution.
3. **Posterior Analysis**: Analyze the posterior samples to estimate parameters, uncertainties, and credible intervals.

PyMC builds on [**Theano**](https://en.wikipedia.org/wiki/Theano_(software)), which allows it to compute gradients and perform symbolic differentiation for more efficient sampling.

## 1.1 Bayesian inference over the mean of a Gaussian using PyMC

In [Lab 1 for this module](https://colab.research.google.com/github/m-caprio/COMP64102-Reasoning-and-Learning-under-Uncertainty-Module/blob/main/New%20Lab%201.ipynb), you computed the posterior distribution over the mean of a Gaussian distribution in closed form using the fact that the Gaussian likelihood and the prior over the mean of the Gaussian likelihood are conjugate. The equation for the mean of the posterior distribution had a closed form.

As a warm up example in the use of PyMC, we will do the same example using the probabilistic programming language. Refer to [Lab 1](https://colab.research.google.com/github/m-caprio/COMP64102-Reasoning-and-Learning-under-Uncertainty-Module/blob/main/New%20Lab%201.ipynb) when reading forward.


Besides installing PyMC, we will also install the [ArViz](https://python.arviz.org/en/stable/) package, which is used for exploratory analysis of Bayesian models.

In [ ]:
!pip install pymc
import pymc as pm
import arviz as az

The following code uses PyMC to obtain samples from the posterior distribution of the mean of the Gaussian

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Observed data (assumed to be generated from a Gaussian distribution)
data = np.array([3.2, 2.8, 3.6, 3.0, 3.1, 2.9, 3.3, 3.5])

# Known standard deviation of the data
sigma = 0.5

# Define the model in PyMC
with pm.Model() as model:
    # Prior for mu (mean) - assume Gaussian prior
    mu_0 = 3.0  # Prior mean
    sigma_0 = 1.0  # Prior standard deviation
    mu = pm.Normal('mu', mu=mu_0, sigma=sigma_0) # Prior

    # Likelihood for the data - assume Gaussian likelihood
    likelihood = pm.Normal('likelihood', mu=mu, sigma=sigma, observed=data) # Likelihood

    # Sample from the posterior distribution
    trace = pm.sample(2000, return_inferencedata=True)

# Extract posterior samples
mu_p_samples = trace.posterior['mu'].values.flatten()

# Compute posterior mean and standard deviation
mu_p = np.mean(mu_p_samples)
sigma_p = np.std(mu_p_samples)

# Print results
print(f"Posterior mean (mu_p): {mu_p:.2f}")
print(f"Posterior standard deviation (sigma_p): {sigma_p:.2f}")

The values for the posterior mean and posterior standard deviation when comparing the exact solution in Lab 1 and the sampling approach using PyMC are the same.

**Question 1 (5 marks)**

What is the fundamental difference between the approach to compute the mean of the posterior distribution in Lab 1 and the approach to compute the mean using PyMC.

*Solution*

Write our answer here


We can now plot the prior, likelihood and the posterior pdf's.

In [ ]:
# Plot prior, likelihood, and posterior
x = np.linspace(2, 4, 1000)  # Range of mu values to plot
prior = norm.pdf(x, mu_0, sigma_0)  # Prior distribution
likelihood = norm.pdf(x, np.mean(data), sigma / np.sqrt(len(data)))  # Likelihood
posterior = norm.pdf(x, mu_p, sigma_p)  # Posterior distribution

plt.figure(figsize=(10, 6))
plt.plot(x, prior, label="Prior", linestyle="--")
plt.plot(x, likelihood, label="Likelihood", linestyle="-.")
plt.plot(x, posterior, label="Posterior", linestyle="-")
plt.title("Bayesian Update of Gaussian Mean (PyMC)")
plt.xlabel("Mean ($\mu$)")
plt.ylabel("Probability Density")
plt.legend()
plt.show()

Two metrics to assess convergence in MCMC are the **effective sample size** (ESS) and the **estimated potential scale reduction** (EPSRC) or **R-hat**.

We can use the [ArViz](https://python.arviz.org/en/stable/) package to compute both quantities.

In [ ]:
# Posterior summary (for more detailed diagnostics)
print(az.summary(trace, var_names=["mu"], hdi_prob=0.95))

**Question 2 (5 marks)**

The value of `r_hat` for the mean is equal to 1.0. Explain what `r_hat` computes and whether 1.0 is a good value. Explain why it is or why it is not a good value.

*Solution*

Write your answer here

## 1.2 Bayesian logistic regression with PyMC

In the previous module in Foundations of Machine Learning, we spent some time on a linear model for classification called **logistic regression**.

In a binary classification problem, let $\mathbf{X}=[\mathbf{x}_1 \cdots \mathbf{x}_N]^\top$ and and $\mathbf{y} =[y_1\cdots y_N]^{\top}$, where $\mathbf{X}$ is the design matrix and each element in $\mathbf{y}$, say $y_n$ indicates to which class the corresponding input vector, $\mathbf{x}_n$, belongs to.

Assumming IID observations, the likelihood for the dataset is given as
\begin{align*}
p(\mathbf{y}|\mathbf{w},\mathbf{X}) = \prod_{n=1}^Np(y_n|\mathbf{w},\mathbf{x}_n)
  =\prod_{n=1}^N\textrm{Ber}(y_n|\sigma(\mathbf{w}^{\top}\mathbf{x}_n)),
\end{align*}
where $\textrm{Ber}(y_n|\mu)$ refers to the Bernoulli distribution with parameter $\mu$, and $\sigma(a)$ is the sigmoid function defined as $\sigma(a)=\frac{1}{1+ \exp(-a)}$.

The maximum-likelihood estimator for $\mathbf{w}$ can be obtained by minimising the negative log-likelihood (or the cross-entropy function) given as

\begin{align*}
NLL(\mathbf{w}) & = - \log p(\mathbf{y}|\mathbf{w},\mathbf{X}) \\
& = - \sum_{n=1}^N\{y_n\log[\sigma(\mathbf{w}^{\top}\mathbf{x}_n)] + (1-y_n)\log[1 - \sigma(\mathbf{w}^{\top}\mathbf{x}_n)] \}.
\end{align*}

In *Bayesian logistic regression*, we put a prior over the weights $\mathbf{w}$. We then compute the posterior distribution
\begin{align*}
p(\mathbf{w}|\mathbf{X}, \mathbf{y}) \propto p(\mathbf{y}|\mathbf{w}, \mathbf{X})p(\mathbf{w}).
\end{align*}
For predicting the class of a text point, $\mathbf{x}_*$, we need to compute the predictive distribution,
\begin{align}
p(y_*|\mathbf{x}_*)=\int \sigma(\mathbf{w}^{\top}\mathbf{x}_*)p(\mathbf{w}|\mathbf{X}, \mathbf{y})d\mathbf{w}.
\end{align}

If we approximate the posterior distribution $p(\mathbf{w}|\mathbf{X}, \mathbf{y})$ with a Gaussian posterior distribution,
\begin{align*}
p(\mathbf{w}|\mathbf{X}, \mathbf{y}) ≈ q(\mathbf{w}|\mathbf{X}, \mathbf{y}) = N(\mathbf{w}|𝛍_{\text{post}}, 𝚺_{\text{post}}),
\end{align*}
it can be shown the predictive distribution can also be approximated as
\begin{align*}
p(y_*=1|\mathbf{x}_*) = \sigma(\kappa (\sigma^2_a)\mu_a),
\end{align*}
where
\begin{align*}
\mu_a &= 𝛍_{\text{post}}^\top\mathbf{x}_*,\\
\sigma_a^2 &= \mathbf{x}^\top_*𝚺_{\text{post}}\mathbf{x}_*,\\
\kappa(\sigma^2) &= \frac{1}{(1+\pi \sigma^2/8)^{\frac{1}{2}}}
\end{align*}


### Bayesian logistic regression for the Iris dataset

To illustrate how Bayesian logistic regression works, we will use a very well known dataset, the [Iris dataset](https://archive.ics.uci.edu/dataset/53/iris). The dataset has three classes and four input features. We will only use two classes.

Let us load the dataset

In [ ]:
!pip install ucimlrepo
from ucimlrepo import fetch_ucirepo

# fetch dataset
iris = fetch_ucirepo(id=53)

# data (as pandas dataframes)
X3 = iris.data.features
y3 = iris.data.targets
# Drop the last class and the last two columns
Xp = X3.drop(X3.index[100:150])
yp = y3.drop(y3.index[100:150])

We now have a reduced dataset that we will split into training and test

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
# We add a vector of ones to the input space in X
X = Xp.to_numpy()
# Add a column of ones to X
y = np.vstack((np.zeros((50, 1)), np.ones((50, 1))))
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=int(1e3))
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

**Question 3 (10 marks)**

Let us define the following prior distribution for $\mathbf{w}$, $p(\mathbf{w})= N(\mathbf{w}|\mathbf{0}_4, 0.001\times \mathbf{I}_4)$, where $\mathbf{0}_4 = [0, 0, 0, 0]^\top$ and $\mathbf{I}_4$ is the identity matrix of dimension four since there are four input features.

Using the training data `X_train` and `y_train`, compute an approximate posterior Gaussian distribution using PyMC for a Bayesian logistic regression model. Use 2000 samples for the sampler and report the mean and the covariance matrix for the approximate Gaussian using the last 1000 samples of one of the chains (**7 marks**)

*Solution*

In [ ]:
# Write your code here.

Compute the accuracy of the linear classifier over the test set using the approximated distribution $p(y_*=1|\mathbf{x}_*) = \sigma(\kappa (\sigma^2_a)\mu_a)$, as explained above (**3 marks**).

*Solution*

In [ ]:
# Write your code here

# 2. Active Learning -- Using data selection via uncertainty to keep models fresh

We can use
uncertainties to figure out whether data should be part of the training data
or not. We can expand on this idea in the context of an area of
machine learning called **active learning**. The promise of active
learning is that a model can learn more eﬀectively on less data if we
have a way to control the type of data it is trained on. Active learning is a way to
guide the learning process and data a model is trained on by
**providing acquisition functions** that can acquire data from a pool of data that is
not part of the training data. By iteratively selecting the right data
from the pool, we can train a model that performs better than if we
had chosen the data from the pool at random.

The active learning approach can be described with the following iterative procedure
```
A model is initially trained using a few samples per class
for iter until a fixed number of iterations:
  compute an acquisition function over the data in the pool
  based on the acquisition function select a sample to include in the  
     training data
  train the model again using the augmented training data
```

In this part of the coursework, we will use results from a fundamental
active learning paper: [Deep Bayesian Active Learning with Image Data
(2017)](https://arxiv.org/abs/1703.02910). We will use the MNIST dataset and train a model on more
and more data, where we select the data points to add to our
training set via an uncertainty method. In this case, we will use **epistemic uncertainty** to select the most informative data points.
Images with high epistemic uncertainty will be added to the traininig dataset in a sequential manner. The epistemic uncertainty can be computed using different methods. You will use **Monte Carlo dropout** and will compare the performance of using epistemic uncertainty to introduce new points in the training data against including new training points just but randomly selecting from a pool of available data.

**IMPORTANT** To reduce the amount of code appearing in the notebook, we have created a set of python modules that include auxiliary functions. You need to make sure when running the notebook that your Google Drive is mounted and the auxiliar Python files can be accessed from the Notebook. On how to allow your Notebook to find the .py files in your Google Drive, see instructions in this [link](https://nimbusintelligence.com/2023/03/import-python-scripts-in-google-colab/)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
# The path here can be different, depending on where you have uploaded the auxiliary Python files config.py, main.py, utils.py, model.py, visualise.py, data.py
sys.path.insert(0,'/content/drive/My Drive/Colab Notebooks')

We start by importing a set of modules

In [ ]:
import dataclasses
from typing import Optional, Tuple, Union, Callable
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import torchvision
from tqdm import tqdm
import json
from pathlib import Path
import gc
import uuid
import matplotlib.pyplot as plt

In [ ]:
!pip install laplace-torch
from laplace import Laplace

## 2.1 Preparing the dataset

We work with the MNIST dataset.

In [ ]:
from data import get_data, prepare_data_loaders

# Get preprocessed MNIST data
data = get_data()

# Create PyTorch DataLoaders
train_loader, test_loader = prepare_data_loaders(data, batch_size=64)

# Function to display a grid of images
def display_mnist_samples(images, labels, grid_size=(3, 3)):
    fig, axes = plt.subplots(grid_size[0], grid_size[1], figsize=(8, 8))

    for i, ax in enumerate(axes.flat):
        if i < len(images):
            # Remove the extra dimension and display the image
            ax.imshow(images[i].squeeze(), cmap='gray')
            # Get the original label (not one-hot encoded)
            label = np.argmax(labels[i])
            ax.set_title(f"Digit: {label}")
            ax.axis('off')

    plt.tight_layout()
    plt.show()

# Display a few samples from the training set
num_samples = 9
indices = np.random.choice(len(data.x_train), num_samples, replace=False)
sample_images = data.x_train[indices]
sample_labels = data.y_train[indices]

# Display the images
display_mnist_samples(sample_images, sample_labels)

# Print some information about the dataset
print(f"Total training samples: {len(data.x_train)}")
print(f"Total test samples: {len(data.x_test)}")

## 2.2 Defining the model

We can now define our model. We will use a small, simple Convolutional Neural Network with dropout as shown below:

```
class MNISTModel(nn.Module):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=4, padding=0),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=4, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.25)
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(3872, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = x.permute(0, 3, 1, 2)
        x = self.conv_layers(x)
        x = torch.flatten(x, 1)
        x = self.fc_layers(x)
        return x

def build_model():
    model = MNISTModel()
    return model

```
This network model has been defined in the file `model.py` and we will call it later.

## 2.3 Defining the uncertainty functions

### 2.3.1 Epistemic or knowledge uncertainty

We will use epistemic uncertainty (also called knowledge
uncertainty) as one of our main uncertainty functions to acquire new
samples. Let us define the function to compute epistemic uncertainty
over our predictions.

We define the followoing terms and notations:
- $\omega$: Model parameters (weights and biases)
- $p(y = c|x, \omega)$: Probability of input $x$ belonging to class $c$ with model parameters $\omega$
- $q_\theta^*(\omega)$: Approximate posterior distribution over model parameters
- $\hat{\omega}_t \sim q_\theta^*(\omega)$: A sample of weights drawn from the approximate posterior
- $\hat{p}_c^t = p(y = c|x, \hat{\omega}_t)$: Predicted probability for class $c$ using sampled weights $\hat{\omega}_t$ in MC dropout
- $T$: Total number of Monte Carlo dropout forward passes
- $C$: Number of classes

### Total Uncertainty

The total predictive uncertainty is captured by the entropy of the expected predictive distribution (Max Entropy, (Shannon, 1948)):

$$H[y|x, D_{\text{train}}] = -\sum_{c} p(y = c|x, D_{\text{train}}) \log p(y = c|x, D_{\text{train}})$$

This can be approximated using $T$ forward passes with MC dropout:

$$H[y|x, D_{\text{train}}] \approx -\sum_{c} \left(\frac{1}{T}\sum_{t=1}^{T} \hat{p}_c^t\right) \log \left(\frac{1}{T}\sum_{t=1}^{T} \hat{p}_c^t\right)$$

### Data Uncertainty (Aleatoric)

Data uncertainty represents the inherent noise in the data, approximated as the expected entropy of individual predictions (Gal, 2016, pp. 48–52):

$$E_{p(\omega|D_{\text{train}})}[H[y|x, \omega]] \approx \frac{1}{T}\sum_{t=1}^{T} \left(-\sum_{c} \hat{p}_c^t \log \hat{p}_c^t\right)$$

### Knowledge Uncertainty (Epistemic)

Knowledge uncertainty is the mutual information between predictions and model posterior, which is the BALD acquisition function (Houlsby et al., 2011) in the paper (which is Total `Uncertainty` - `Data Uncertainty`) :

$$I[y, \omega|x, D_{\text{train}}] = H[y|x, D_{\text{train}}] - E_{p(\omega|D_{\text{train}})}[H[y|x, \omega]]$$

This quantity represents the uncertainty in the model parameters, which can be reduced by acquiring more data. This is why it serves as an effective acquisition function for active learning.



Knowledge uncertainty will be used as acquisition function in the active learning loop. To compute it, we first need the weights obtained through MC dropout.

`get_mc_predictions` uses MC droupout to generate the necessary prediction distribution (multiple predictions per sample) that the uncertainty functions need as input. It creates the `preds` tensor $\hat{p}_c^t$ with shape (n_samples, n_mc_passes, n_classes) that is directly passed to `total_uncertainty`, `data_uncertainty`, and `knowledge_uncertainty`. Without these stochastic samples $\hat{p}_c^t$ from dropout, we could not calculate these different uncertainty types for active learning sample selection.

In [ ]:
def get_mc_predictions(model: nn.Module, n_iter: int, x_train: torch.Tensor) -> torch.Tensor:
    device = next(model.parameters()).device
    preds = []
    # Set model to training mode to enable dropout
    model.train()
    with torch.no_grad():
        for _ in tqdm(range(n_iter)):
            # Split data into batches
            preds_iter = []
            for batch in torch.chunk(x_train, chunks=6):
                batch = batch.to(device)
                # Ensure input is float and has correct shape (add channel dim if needed)
                if len(batch.shape) == 3:
                    batch = batch.unsqueeze(1)
                preds_iter.append(model(batch))
            # Concatenate batch predictions
            preds.append(torch.cat(preds_iter, dim=0))
    # Stack predictions and move axes to get (n_images, n_predictions, n_classes)
    preds = torch.stack(preds, dim=1)
    return preds

**Question 3 (10 marks)**

Write the code for the functions `data_uncertainty` and `total_uncertainty` below

*Solution*

In [ ]:
def total_uncertainty(preds: torch.Tensor, epsilon: float = 1e-10) -> torch.Tensor:
    #
    #
    return

def data_uncertainty(preds: torch.Tensor, epsilon: float = 1e-10) -> torch.Tensor:
    #
    #
    return

def knowledge_uncertainty(preds: torch.Tensor, epsilon: float = 1e-10) -> torch.Tensor:
    # Knowledge uncertainty is the difference between total and data uncertainty
    return total_uncertainty(preds, epsilon) - data_uncertainty(preds, epsilon)

The acquisition function that uses the knowledge uncertainty function can be summarised in three steps
1. Obtain an ensemble of predictions via MC dropout (done by `get_mc_predictions`).
2. Compute the knowledge uncertainty values over this prediction
ensemble (use `knowledge_uncertainty` function).
3. Sort the uncertainty values, get their index and return the
indices of the data in the pool with the highest epistemic
uncertainty.

In [ ]:
def acquire_knowledge_uncertainty(
    x_train: torch.Tensor, n_samples: int, model: nn.Module, n_iter: int, *args, **kwargs
):
    # Get Monte Carlo predictions
    preds = get_mc_predictions(model, n_iter, x_train)
    # Compute knowledge uncertainty
    ku = knowledge_uncertainty(preds)
    # Get indices of samples with highest uncertainty
    _, indices = torch.topk(ku, n_samples)
    return indices.cpu().numpy()

### 2.3.2 Laplace uncertainty

Rather than using MC dropout to compute samples on the parameters of the neural network, we can use a Laplace approximation, which approximates the posterior distribution of the parameters using a Gaussian distribution.

The predictive distribution under a Bayesian approach (Gal, 2016) can be written as:

$$p(y=c|x, D_{train}) = \int p(y=c|x, \omega)p(\omega|D_{train})d\omega.$$

With Laplace approximation, we approximate the posterior $p(\omega|D_{train})$ with a **Gaussian distribution** $N$ centered at the MAP estimate:

$$p(\omega|D_{train}) \approx N(\omega|\omega_{MAP}, \Sigma_{post}),$$

where:
- $\omega_{MAP}$ is the maximum a posteriori estimate of the parameters
- $\Sigma_{post}$ is the covariance matrix computed at $\omega_{MAP}$.

We can then use this posterior distribution and the probit likelihood to approximate the posterior probabilities over the classes.

Computing the Laplace approximation for a posterior distribution of a neural network is cumbersome in general. Here, we use a recent package known as [Laplace that allows to compute the posterior distribution effortlessly](https://github.com/AlexImmer/Laplace).

### Uncertainty Measure for Acquisition when using the Laplace approximation

We use Max Entropy (Shannon, 1948) as an uncertainty measure in this case:

$$H[y|x, D_{train}] := -\sum_{c} p(y=c|x, D_{train}) \log p(y=c|x, D_{train})$$

This acquisition function selects pool points that maximize the predictive entropy.

The acquisition function that uses the max entropy with Laplace approximation can be summarised in three steps
1. Obtain the predictions under the posterior distribution via Laplace approximation.
2. Compute the max entropy values for the datapoints in the pool.
3. Sort the uncertainty values, get their index and return the
indices of the data in the pool with the highest epistemic
uncertainty.

**Question 4 (10 marks)**

The following acquisition function implements the three steps above. Write the code for steps 1 and 2.

*Solution*

In [ ]:
def acquire_laplace_uncertainty(
    unlabeled_tensor: torch.Tensor, n_samples: int, la_model: Laplace) -> np.ndarray:
    try:
        # Step 1: Predict probabilities with error handling
        #
        # Convert to numpy with checks
        try:
            preds_np = preds.numpy()
        except Exception:
            preds_np = preds.detach().cpu().numpy()

        # Step 2: Entropy calculation
        #
        #

        # Step 3: Select indices with highest uncertainty
        indices = np.argsort(uncertainty)[-n_samples:]
        return indices

    except Exception as e:
        print(f"Laplace Acquisition Error: {e}")
        # Fallback to random sampling
        return np.random.choice(len(unlabeled_tensor), n_samples, replace=False)

### 2.3.3 Random selection

The simplest approach to select a datapoint from the pool is to do it randomly

In [ ]:
def acquire_random(x_train: torch.Tensor, n_samples: int, *args, **kwargs) -> np.ndarray:
    # Randomly select samples
    return torch.randperm(len(x_train))[:n_samples].numpy()

### 2.3.4 A battery of acquisition functions

We can group the three acquisition functions above in a function,

In [ ]:
def acquisition_factory(acquisition_type: str) -> Callable:
    if acquisition_type == "laplace_uncertainty":
        return acquire_laplace_uncertainty
    if acquisition_type == "knowledge_uncertainty":
        return acquire_knowledge_uncertainty
    elif acquisition_type == "random":
        return acquire_random
    else:
        raise ValueError(f"Acquisition type {acquisition_type} not supported")

Now that we have defined our acquisition functions, we are ready
to define the loop that runs the active learning iterations.

## 2.4 Main Loop for Active Learning

We will start our dataset with 20 samples until we reach a total of 500 samples. Each model will be trained for 25 epochs and per iteration, we acquire 10 samples. To obtain our MC dropout predictions, we will run over
our full training set (minus the already acquired samples) 48 times.

It will take you approx. 1 hour to finish the active learning loop using standard colab T4 GPU.

In [ ]:
import main
main.acquisition_factory= acquisition_factory #Put your acquisition functions inside the main module
from main import Active_learning

In [ ]:
# Run Active Learning on Laplace Uncertainty Acquisition
accuracies_l, added_indices_l = Active_learning(
    acquisition_type="laplace_uncertainty",
    n_epochs=50,
    n_samples_per_iter=10,
    initial_n_samples=20,
    n_total_samples=500,
    use_wandb=False
)

# Run Active Learning on Random Acquisition
accuracies_r, added_indices_r = Active_learning(
    acquisition_type="random",
    n_epochs=50,
    n_samples_per_iter=10,
    initial_n_samples=20,
    n_total_samples=500,
    use_wandb=False
)

# Run Active Learning on Knowlege Uncertainty Acquisition
accuracies_k, added_indices_k = Active_learning(
   acquisition_type="knowledge_uncertainty",
    n_epochs=50,
    n_samples_per_iter=10,
    initial_n_samples=20,
    n_total_samples=500,
    use_wandb=False
)

### 2.5  Visualise the Results

Now that we have our loop, we can inspect the results of this
process. We will use seaborn and matplotlib to visualize our
results. The main result we are interested in is the test accuracy over time
for both the models trained with a random acquisition function and
the models trained with data acquired via knowledge uncertainty.
To visualize this, we define `ActiveLearningVisualizer` that loads the results and
then returns a plot that shows the accuracy per active learning
iteration cycle.

We can then use this function to plot the results for all acquisition
functions:

In [ ]:
from visualise import ActiveLearningVisualizer
import matplotlib.pyplot as plt

# Initialize visualizer
visualizer = ActiveLearningVisualizer(output_dir="output", samples_per_iter=10, initial_samples=20)

# Compare different acquisition strategies
# IMPORTANT: change the uuids according to the ones assigned to each of the models trained
fig_acc, fig_img, imgs = visualizer.compare_from_uuids(
    uuids=["f911b431-3db2-422f-b2fe-fe5908c38178",
            "3431804b-e582-48ba-a8e2-872c87c63146",
            "2dcb6f4f-495e-4b5c-b50c-03daedcd1ef2"],
    acquisition_types=["laplace_uncertainty", "random", "knowledge_uncertainty"]
)

plt.show()

**Question 5 (10 marks)**

- What can you conclude from the figure that shows the Accuracy of the different acquisition strategies? (**5 marks**). Use a single sentence.

*Solution*

Write your answer here

- What can you conclude when comparing the samples of the digits chosen for training by the different Acquisition strategies? (**5 marks**). Use a single sentence.

*Solution*

Write your answer here